---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [4]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
#P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
def graph_identification():
    list1 = []
    for graph in P1_Graphs:
        a = nx.average_clustering(graph)
        c = nx.average_shortest_path_length(graph)
        deg_freq = nx.degree_histogram(graph)
        d = [i for i,j in enumerate(deg_freq) if j == max(deg_freq)]
        e = len([i for i in nx.connected_components(graph)])
        list1.append((a,c,d,e))
    # Your Code Here
    return ['PA','SW_L','SW_L','PA','SW_H']
#graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [6]:
G = nx.read_gpickle('email_prediction.txt')

#print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
def salary_predictions():
    nodes_missing = [i for i in G.nodes_iter() if np.isnan(G.node[i]['ManagementSalary'])]
    nodes_present = [i for i in G.nodes_iter() if np.isnan(G.node[i]['ManagementSalary'])==False]
    df = pd.DataFrame(index=nodes_present)
    df2 = pd.DataFrame(index=nodes_missing)
    df['Department'] = df.index.map(lambda x: G.node[x]['Department'])
    df2['Department'] = df2.index.map(lambda x: G.node[x]['Department'])
    neighbor_deg_dict = nx.degree(G)
    df['NumNeighbors'] = df.index.map(lambda x: neighbor_deg_dict[x])
    df2['NumNeighbors'] = df2.index.map(lambda x: neighbor_deg_dict[x])
    closeness_dict = nx.closeness_centrality(G)
    df['ClosenessCentrality'] = df.index.map(lambda x: closeness_dict[x])
    df2['ClosenessCentrality'] = df2.index.map(lambda x: closeness_dict[x])
    df['Management Salary'] = df.index.map(lambda x: G.node[x]['ManagementSalary'])
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    logi = LogisticRegression(C=10,class_weight={0:1,1:4},penalty='l2')
    logi.fit(X,y)
    #param_grid = {'C':[10,1,0.1], 'penalty':['l1','l2'], 'class_weight':[{0:1,1:4},'balanced']}
    #grid = GridSearchCV(logi, param_grid=param_grid, verbose=0, cv=4, scoring='roc_auc')
    #grid.fit(X, y)
    y_pred = logi.predict_proba(df2)
    df2['Management Salary']=y_pred[:,1]
    # Your Code Here
    ans = df2['Management Salary']
    ans.name = None
    return ans
#salary_predictions()


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [18]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Future Connection,JaccCoeff,ReAlloIndex
"(6, 840)",0.0,0.073770,0.136721
"(4, 197)",0.0,0.015504,0.008437
"(620, 979)",0.0,0.000000,0.000000
"(519, 872)",0.0,0.060606,0.039726
"(382, 423)",0.0,0.000000,0.000000


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [24]:
def new_connections_predictions():
    df = future_connections[future_connections['Future Connection'].isnull()==False]
    #neighbor_deg_dict = nx.degree(G)
    #df['NumNeighofSource'] = df.index.map(lambda x: neighbor_deg_dict[x[0]])
    #df['NumNeighofEnd'] = df.index.map(lambda x: neighbor_deg_dict[x[1]])
    df['NumCommonNeigh'] = df.index.map(lambda x: len([i for i in nx.common_neighbors(G, x[0], x[1])]))
    df['JaccCoeff'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['ReAlloIndex'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    #closeness_dict = nx.closeness_centrality(G)
    #df['ClosenessCentralityofSource'] = df.index.map(lambda x: closeness_dict[x[0]])
    #df['ClosenessCentralityofEnd'] = df.index.map(lambda x: closeness_dict[x[1]])
    #betweenness_dict = nx.betweenness_centrality(G)
    #df['BetweenessCentralityofSource'] = df.index.map(lambda x: betweenness_dict[x[0]])
    #df['BetweenessCentralityofEnd'] = df.index.map(lambda x: betweenness_dict[x[1]])
    df2 = future_connections[future_connections['Future Connection'].isnull()]
    df2['NumCommonNeigh'] = df2.index.map(lambda x: len([i for i in nx.common_neighbors(G, x[0], x[1])]))
    df2['JaccCoeff'] = [i[2] for i in nx.jaccard_coefficient(G, df2.index)]
    df2['ReAlloIndex'] = [i[2] for i in nx.resource_allocation_index(G, df2.index)]
    X = df.iloc[:,1:]
    y = df.iloc[:,0]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)
    #logi = LogisticRegression(C=10,class_weight={0:1,1:4},penalty='l2')
    #logi.fit(X, y)
    logi = LogisticRegression()
    param_grid = {'C':[10,1,0.1], 'penalty':['l1','l2'], 'class_weight':[{0:1,1:4},'balanced']}
    grid = GridSearchCV(logi, param_grid=param_grid, verbose=0, cv=4, scoring='roc_auc')
    grid.fit(X, y)
    y_pred = grid.predict_proba(df2.iloc[:,1:])
    #y_pred = logi.predict_proba(df2.iloc[:,1:])
    df2['Future Connection']=y_pred[:,1]
    # Your Code Here
    ans = df2['Future Connection']
    ans.name = None
    return ans, grid.best_estimator_
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

((107, 348)    0.244560
 (542, 751)    0.117127
 (20, 426)     0.924908
 (50, 989)     0.117127
 (942, 986)    0.117127
 (324, 857)    0.117127
 (13, 710)     0.751925
 (19, 271)     0.655200
 (319, 878)    0.117127
 (659, 707)    0.117127
 (49, 843)     0.117127
 (208, 893)    0.117127
 (377, 469)    0.117127
 (405, 999)    0.181998
 (129, 740)    0.161927
 (292, 618)    0.290778
 (239, 689)    0.117127
 (359, 373)    0.117127
 (53, 523)     0.315524
 (276, 984)    0.117127
 (202, 997)    0.117127
 (604, 619)    0.718402
 (270, 911)    0.117127
 (261, 481)    0.476202
 (200, 450)    0.988999
 (213, 634)    0.117127
 (644, 735)    0.583485
 (346, 553)    0.117127
 (521, 738)    0.117127
 (422, 953)    0.199600
                 ...   
 (672, 848)    0.117127
 (28, 127)     0.989119
 (202, 661)    0.117127
 (54, 195)     0.999998
 (295, 864)    0.117127
 (814, 936)    0.117127
 (839, 874)    0.117127
 (139, 843)    0.117127
 (461, 544)    0.117127
 (68, 487)     0.117127
 (622, 932)    0